In [1]:
from main_test import get_estimated_means, get_order_statistic_pdfs, transform_covariates
import numpy as np
from scipy import stats, integrate, interpolate, optimize
import matplotlib.pyplot as plt
import sympy as sp

In [2]:
covariates = [[cov1, cov2] for cov1, cov2 in zip(np.random.uniform(0, 1, size=40), np.random.uniform(0, 1, size=40))]
bids = []
bids.append([stats.norm.rvs(loc=2*x[0], scale=1, size=10) for x, _ in zip(covariates[:20], range(20))])
bids.append([stats.norm.rvs(loc=2*x[0], scale=1, size=5) for x, _ in zip(covariates[20:], range(20))])
bids = np.ravel(bids)
# covariates = transform_covariates(covariates, 20)

/Users/liumukun/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:1859: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asanyarray(a).ravel(order=order)


In [3]:
pdfs, pdfs_delta = get_order_statistic_pdfs(bids, covariates)

expected_upper = {}
expected_lower = {}
unique_covariates = [list(x) for x in set(tuple(x) for x in covariates)]

for covariate in unique_covariates:
    lower, upper = get_estimated_means(pdfs, pdfs_delta, covariate, (-2,2))
    expected_upper[f"{covariate}"] = upper
    expected_lower[f"{covariate}"] = lower

smooth upper bound for -2.0: 0.0012651712774521537
smooth upper bound for -1.9595959595959596: 0.001455581683599288
smooth upper bound for -1.9191919191919191: 0.0016778229058495553
smooth upper bound for -1.878787878787879: 0.00189109586379075
smooth upper bound for -1.8383838383838385: 0.00213256812694393
smooth upper bound for -1.797979797979798: 0.002540423702011624
smooth upper bound for -1.7575757575757576: 0.0028329806092983138
smooth upper bound for -1.7171717171717171: 0.003201616380543703
smooth upper bound for -1.6767676767676767: 0.0036432308450772334
smooth upper bound for -1.6363636363636362: 0.004085758468041488
smooth upper bound for -1.595959595959596: 0.00458876261708821
smooth upper bound for -1.5555555555555556: 0.005168508072280527
smooth upper bound for -1.5151515151515151: 0.005807752986182418
smooth upper bound for -1.4747474747474747: 0.006507616096785042
smooth upper bound for -1.4343434343434343: 0.007274709205026537
smooth upper bound for -1.393939393939394:

smooth lower bound for 0.020202020202020332: 0.11339665776330272
smooth lower bound for 0.060606060606060996: 0.12011244339814313
smooth lower bound for 0.10101010101010122: 0.1302585542348676
smooth lower bound for 0.14141414141414144: 0.13797296614612148
smooth lower bound for 0.1818181818181821: 0.14426999059072135
smooth lower bound for 0.22222222222222232: 0.15204238514775983
smooth lower bound for 0.262626262626263: 0.37376303786292653
smooth lower bound for 0.3030303030303032: 0.3822523640993395
smooth lower bound for 0.3434343434343434: 0.3893144517691003
smooth lower bound for 0.3838383838383841: 0.6842772772820933
smooth lower bound for 0.4242424242424243: 0.693138873753512
smooth lower bound for 0.464646464646465: 0.43950099069907905
smooth lower bound for 0.5050505050505052: 0.45484109805741924
smooth lower bound for 0.5454545454545459: 0.47332736269429265
smooth lower bound for 0.5858585858585861: 0.4894460959669822
smooth lower bound for 0.6262626262626263: 0.503713663956

/Users/liumukun/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1695: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return integrate.quad(self._mom_integ1, 0, 1, args=(m,)+args)[0]


ValueError: too many values to unpack (expected 2)

In [ ]:
def get_loss_function(covariates, upper_dict, lower_dict, a, beta):
    _sum = 0
    for covariate in covariates:
        upper = upper_dict[f"{covariate}"]
        lower = lower_dict[f"{covariate}"]
        if beta*covariate[0] > upper:
            _sum += (beta*covariate[0]-upper)**2
        if beta*covariate[0] < lower:
            _sum += (beta*covariate[0]-lower)**2
    return _sum

def loss_function(c):
    a,b = c
    return get_loss_function(covariates, expected_upper, expected_lower, a, b)

b_hat = optimize.brute(loss_function, ranges=[(0,2),(-1,1)])
# interval_lower = optimize.newton(lambda b_l: loss_function(b_l)-loss_function(b_hat)-10, b_hat-0.1)
# interval_upper = optimize.newton(lambda b_u: loss_function(b_u)-loss_function(b_hat)-10, b_hat+0.1)

# print(b_hat, interval_lower, interval_upper)

In [ ]:
from scipy import integrate

vals = np.linspace(0,3,num=20)

for idx, func in enumerate(pdfs["5"]):
    plt.plot(vals, [cdf(1, lambda x: integrate.quad(func([0,0]), 0, v)[0], idx+1, 7) for v in vals], label=idx)

plt.legend()

In [ ]:
import matplotlib.pyplot as plt

r = np.linspace(-2,4,num=200)

print(b_hat)
plt.plot(r, [loss_function((0,b)) for b in r])

In [ ]:
from random import choices

def bootstrap(bids, covariates, repetition):
    
    estimates_lower = []
    estimates_upper = []
    data_pairs = [(bid, covariate) for bid, covariate in zip(bids, covariates)]
    
    for _ in range(repetition):
        resampled = choices(data_pairs, k=len(bids))

        bids, covariates = resampled
        pdfs, pdfs_delta = get_order_statistic_pdfs(bids, covariates)
        
        expected_upper = {}
        expected_lower = {}
        
        for covariate in np.unique(covariates):
            lower, upper = get_estimated_means(pdfs, pdfs_delta, covariate)
            expected_upper[f"{covariate}"] = upper
            expected_lower[f"{covariate}"] = lower
        
        estimate = optimize.brute(lambda b: loss_function(covariates, expected_upper, expected_lower, b), ranges=[(0,2)])
        interval_lower = optimize.newton(lambda b_l: loss_function(b_l)-loss_function(estimate)-10, estimate-0.1)
        interval_upper = optimize.newton(lambda b_l: loss_function(b_l)-loss_function(estimate)-10, estimate+0.1)
        
        estimates_lower.append(interval_lower)
        estimates_upper.append(interval_upper)
        
    return (estimates_lower, estimates_upper)
    

In [ ]:
bootstrap(bids, covariates, 1)